In [8]:
# 1. Install necessary libraries
!pip install -q -U google-generativeai pandas openpyxl

# 2. Import libraries and set up API key
import google.generativeai as genai
import pandas as pd
from google.colab import userdata
from google.colab import drive
import time

# Get the API key from Colab secrets
# IMPORTANT: Add your GOOGLE_API_KEY to Colab secrets.
# Go to the "🔑" icon in the left sidebar, click "Add new secret",
# enter "GOOGLE_API_KEY" as the Name and paste your API key as the Value.
# Make sure "Notebook access" is enabled.
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
except userdata.SecretNotFoundError:
    print("Error: GOOGLE_API_KEY not found in Colab secrets.")
    print("Please add your GOOGLE_API_KEY to Colab secrets using the key icon in the left sidebar.")
    # Exit or handle the error as appropriate for your script
    exit() # Exit the script if the API key is not found


# 3. Mount Google Drive
drive.mount('/content/drive')

# 4. Read the Excel file
# IMPORTANT: Replace 'your_file.xlsx' with the actual name of your file.
# Make sure the file is in the root of your Google Drive or provide the correct path.
file_path = '/content/drive/My Drive/ESG - annotated AN - short.xlsx'
try:
    df = pd.read_excel(file_path)
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
    print("Please make sure the file name is correct and it is uploaded to your Google Drive.")
    # Create a dummy dataframe for demonstration if the file is not found
    df = pd.DataFrame({'text': ['Gemini is a powerful AI.', 'I am not happy with the service.']})


# 5. Function to get sentiment from Gemini
def get_gemini_company_sentiment(row):
    """Analyzes the sentiment of a given text towards a specific company using the Gemini API."""
    text = row['text']
    company = row['company']
    try:
        model = genai.GenerativeModel('gemini-2.0-flash')
        response = model.generate_content(f'Analyze the sentiment of this text towards the company "{company}" and return only one word: "Positive", "Negative", or "Neutral". Text: "{text}"')
        # A short delay to avoid overwhelming the API
        time.sleep(4)
        return response.text.strip()
    except Exception as e:
        return f"Error: {e}"

# 6. Iterate through rows and get sentiment
# Create a new column for the sentiment towards the company
df['SentimentGemini_company'] = df.apply(get_gemini_company_sentiment, axis=1)

# 7. Save the output to a new Excel file
output_path = '/content/drive/My Drive/ESG - annotated AN - short with Sentiment - company.xlsx'
df.to_excel(output_path, index=False)

print("\nSentiment analysis complete!")
print(f"The output has been saved to: {output_path}")

# Display the first 5 rows of the new dataframe
print("\nPreview of the output file:")
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Sentiment analysis complete!
The output has been saved to: /content/drive/My Drive/ESG - annotated AN - short with Sentiment - company.xlsx

Preview of the output file:
   Column1                      id  \
0        0  CLASSLA-web.sl.2068987   
1        1  CLASSLA-web.sl.3237041   
2        2  CLASSLA-web.sl.1371033   
3        3   CLASSLA-web.sl.938966   
4        4  CLASSLA-web.sl.2226581   

                                               title  \
0  Fajfar: Želimo si čim več infrastrukturnih pro...   
1  19. skupščina Telekoma Slovenije - uspešna pri...   
2  Telekom s telemedicinsko rešitvijo za spremlja...   
3              Vožnja na plin je cenejša - RTVSLO.si   
4  VIDEO: Slovenske železnice organizirale pogozd...   

                                                text                   media  \
0  Fajfar: Želimo si čim več infrastrukturnih pro...   